# Bringing in LangChain for our Prompt Engineering

After gaining experience and skill with using single prompts and API use, we can start building this out even more by building prompt pipelines.

We'll move from single prompts to prompt pipelines using LangChain (in tandam with our API gateway that uses the Chat Completions API).

By the end, we'll:
- Call an LLM via LangChain
- Turn a raw prompt into a reusable prompt template
- Build a simple chain: `prompt -> model -> output parser`
- Build a small two-step pipeline: `reasoning -> short final answer`

## Initialize our LangChain object

In this notebook we use LangChain `ChatOpenAI`, but we point it at a gateway that speaks the OpenAI style `chat.completions` API.

Key idea:
- the gateway exposes a `POST /v1/chat/completions` endpoint
- we pass `base_url` and `api_key` for that gateway
- we set `use_responses_api=False` so LangChain uses chat.completions instead of the newer responses API

You can adapt the base URL and model name to match other setups as you wish.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from IPython.display import display, Markdown

import os

NRP_TOK = os.environ.get('NRP_TOK')
nrp_llm_url = "https://ellm.nrp-nautilus.io/v1"

llm = ChatOpenAI(model = 'gpt-oss',
                 api_key = NRP_TOK,
                 base_url = nrp_llm_url,
                 # use_responses_api=False forces the classic chat.completions endpoint
                 use_responses_api=False,
                 temperature = 0.3)

## From single prompts to reusable building blocks

Prompt engineering usually starts with one off strings such as `Explain X in a friendly way`.

But as soon as we want to:
- reuse the same style with different inputs
- add multiple steps, for example draft -> critique -> summarise
- mix in tools or retrieval

we end up needing more structure.

LangChain gives us:
- models: `ChatOpenAI`
- prompt templates: parameterised prompts with variables
- chains: composable flows, for example `prompt -> model -> parser -> next step`


In [ ]:
# Baseline: directly pass a single prompt string to the model.

response = llm.invoke('In 2–3 sentences, what is LangChain, in simple terms?')
print(response.content)

## Prompt templates

Instead of hard coding every prompt as a plain string, we can define a template once and fill in variables.

Benefits for prompt engineers:
- reuse: same structure, different inputs
- experimentation: tweak one template, not many copies
- separation of concerns: prompt design versus application logic

We will use `ChatPromptTemplate`, which is tailored for chat style prompts.


In [ ]:
# Define a reusable prompt template for explanations.

explain_prompt = ChatPromptTemplate.from_template(
    '''
    You are a helpful AI tutor.

    Explain the concept of {topic} to a beginner
    in a {tone} tone, for example friendly, formal, or playful.
    Use short paragraphs and concrete examples.
    '''
)

In [ ]:
explain_prompt

In [ ]:
explain_prompt.input_variables

In [ ]:
explain_prompt.invoke({'topic': 'LangChain', 'tone': 'friendly'})

In [ ]:
# Parse the model response as a simple string.
output_parser = StrOutputParser()

Building our chain: prompt -> model -> output parser

In [ ]:
explain_chain = explain_prompt | llm | output_parser

**Friendly** explanation of vector databases:

In [ ]:
r = explain_chain.invoke(
    {'topic': 'vector databases', 
     'tone': 'friendly'}
)

display(Markdown(r))

**Formal** explanation of vector databases:

In [ ]:
r = explain_chain.invoke(
    {'topic': 'vector databases', 
     'tone': 'formal'}
)

display(Markdown(r))

---

## From single chain to a tiny pipeline

Prompt engineering often benefits from staging:
1. let the model reason in detail
2. then summarise that reasoning for the user

We will build:
- a first chain that generates detailed step by step reasoning
- a second step that compresses that into a short, user friendly answer

LangChain lets us compose these steps with the LangChain Expression Language using the `|` operator.


### Step 1: a prompt that asks the model to think step by step.

In [ ]:
reasoning_prompt = ChatPromptTemplate.from_template(
    '''
    You are a careful reasoning assistant.

    Think step by step to answer the user question.
    Show your reasoning explicitly, with numbered steps.

    Question: {question}
    '''
)

In [ ]:
reasoning_chain = reasoning_prompt | llm | StrOutputParser()

Test it out once:

In [ ]:
reasoning_example = reasoning_chain.invoke(
    {'question': 'Why are large language models called large?'}
)

display(Markdown(reasoning_example))

### Step 2: a prompt that summarises detailed reasoning into a short answer.

In [ ]:
summarize_prompt = ChatPromptTemplate.from_template(
    '''
    You are a friendly assistant.

    You are given a user question and some detailed reasoning.
    Your task is to write a clear, concise answer for the user
    in 2 to 3 sentences, without showing the intermediate steps.

    Question: 
    {question}

    Reasoning:
    {reasoning}

    Final answer, with no preamble, just the answer:
    '''
)

summarize_chain = summarize_prompt | llm | StrOutputParser()

### Combine both steps into a single chain.
* Input: a question string.
* Internal:
  * RunnablePassthrough carries the original question through
  * reasoning_chain produces the reasoning text
* Output: a short final answer.



In [ ]:
two_step_chain = (
    {
        'question': RunnablePassthrough(),
        'reasoning': reasoning_chain,
    }
    | summarize_prompt
    | llm
    | StrOutputParser()
)

question = 'Why do we use prompt templates instead of plain strings?'
final_answer = two_step_chain.invoke(question)

print('Question:', question)
print('\nFinal answer:\n', final_answer)

We have now:
1. called a chat model via LangChain
2. turned a raw prompt into a reusable `ChatPromptTemplate`
3. built a simple chain: `prompt -> model -> parser`
4. composed a two step pipeline: detailed reasoning then a short answer